In [1]:
import os

In [2]:
%pwd

'/Users/mananpatel/Documents/vscode/nlp/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/mananpatel/Documents/vscode/nlp/Text-Summarizer'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [6]:
#configuration manager from src

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
  def __init__(
      self, 
      config_file_path = CONFIG_FILE_PATH,
      params_file_path = PARAMS_FILE_PATH):
    
    self.config = read_yaml(config_file_path)
    self.params = read_yaml(params_file_path)
    create_directories([self.config.artifacts_root])

  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion

    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir = config.root_dir,
      source_URL = config.source_URL,
      local_data_file = config.local_data_file,
      unzip_dir = config.unzip_dir
    )

    return data_ingestion_config
    

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
#update components

class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

  def download_file(self):

    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url = self.config.source_URL,
        filename = self.config.local_data_file
      )
      logger.info(f"{filename} downloading! with the following info: \n{headers}")
    
    else:
      logger.info(f"File already exists! of size : {get_size(Path(self.config.local_data_file))}")

  def extract_zip_file(self):
    """
    zip_file path : str
    Extracts the zip file into the data directory
    Returns: None
    """
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
      zip_ref.extractall(unzip_path)
      logger.info(f"{self.config.local_data_file} unzipped to {unzip_path}")

In [10]:
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config = data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e


[2024-02-18 22:06:07,833]: INFO: common: yaml file : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-02-18 22:06:07,835]: INFO: common: yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-02-18 22:06:07,836]: INFO: common: Created directory: artifacts]
[2024-02-18 22:06:07,836]: INFO: common: Created directory: artifacts/data_ingestion]
[2024-02-18 22:06:09,956]: INFO: 1353919789: artifacts/data_ingestion/samsumdata.zip downloading! with the following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6BC2:0273:F9DDC9:1385419:65D2C